In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from tyssue.generation import generate_ring
from tyssue.draw.plt_draw import quick_edge_draw
from tyssue.solvers.sheet_vertex_solver import Solver
from tyssue.utils import testing
from tyssue.io import hdf5

#from tyssue_taylor.adjusters.adjust_annular import adjust_tensions 
from tyssue_taylor.models.annular import AnnularGeometry as geom
from tyssue_taylor.models.annular import model


In [ ]:
Nf = 12
R_in = 1
R_out = 2

organo = generate_ring(Nf, R_in, R_out)

geom.update_all(organo)
specs = {
    'face':{
        'is_alive': 1,
        'prefered_area': organo.face_df.area.mean()*0.9,
        'area_elasticity': 1,},
    'edge':{
        'ux': 0.,
        'uy': 0.,
        'uz': 0.,
        'line_tension': 1e-2,
        'is_active': 1
        },
    'vert':{
        'adhesion_strength': 0.,
        'x_ecm': 0.,
        'y_ecm': 0.,
        'is_active': 1
        },
    'settings': {
        'lumen_elasticity': 1.,
        'lumen_prefered_vol': organo.settings['lumen_volume'],
        'lumen_volume': organo.settings['lumen_volume']
        }
    }
organo.update_specs(specs, reset=True)
geom.update_all(organo)


organo.vert_df[['x_ecm', 'y_ecm']] = organo.vert_df[['x', 'y']]

organo.vert_df.loc[organo.basal_verts, 'adhesion_strength'] = 1e-3

    

res = Solver.find_energy_min(organo, geom, model,
                             minimize={'options': {'gtol': 1e-4, 'ftol': 1e-4}})
print(res.success)
print(res.message)
fig, ax = quick_edge_draw(organo)
ax.plot(*organo.vert_df[['x_ecm', 'y_ecm']].values.T, 'o')

In [ ]:
tension_noise = np.random.normal(1, 0.15, size=3*Nf).clip(min=0)
area_noise = np.random.normal(1, 0.15, size=Nf).clip(min=0.1)
organo.face_df['prefered_area'] *= area_noise

organo.edge_df.loc[: Nf-1, 'line_tension'] *= 4*tension_noise[: Nf]
organo.edge_df.loc[Nf: 2*Nf-1, 'line_tension'] *= 2*tension_noise[Nf: 2*Nf]

organo.edge_df.loc[2*Nf:3*Nf-1, 'line_tension'] *= tension_noise[2*Nf:]
organo.edge_df.loc[3*Nf: , 'line_tension'] *= tension_noise[2*Nf:]



In [ ]:
res = Solver.find_energy_min(organo, geom, model,
                             minimize={'options': {'gtol': 1e-4, 'ftol': 1e-4}})
print(res.success)
print(res.message)
initial = organo.copy()
fig, ax = quick_edge_draw(organo)
ax.plot(*organo.vert_df.loc[organo.basal_verts,
                            ['x_ecm', 'y_ecm']].values.T, 'o')


In [ ]:
fig, ax = plt.subplots()

estim = organo.edge_df.line_tension.mean() * organo.edge_df.length.mean()/ organo.edge_df.length
ax.plot(organo.edge_df.length, estim, 'd', alpha=0.2)

ax.plot(organo.edge_df.length, organo.edge_df.line_tension, 'o', alpha=0.3)


In [ ]:
initial_guess = (initial.edge_df.length 
                 * initial.edge_df.line_tension).mean() / initial.edge_df.length
initial_guess = initial_guess.values[: 3*Nf]
res = adjust_tensions(organo, initial_guess)

In [ ]:
res.message

In [ ]:
fig, ax = quick_edge_draw(organo)
fig, ax = quick_edge_draw(initial, ax=ax)

In [ ]:
fig, ax = plt.subplots()
ax.plot(initial.edge_df.line_tension, organo.edge_df.line_tension, 'o')
ax.plot([initial.edge_df.line_tension.min(), initial.edge_df.line_tension.max()],
        [initial.edge_df.line_tension.min(), initial.edge_df.line_tension.max()], 'k-')

ax.set(
    xlabel='Inital line tensions',
    ylabel='Computed line tensions')
